**Great — here’s your modified script with the adjustments to resize chart images to a long-side target in the ~640–768 pixel range (preserving aspect ratio), along with prints of original/new image sizes and total inference time.**

In [1]:
# -*- coding: utf-8 -*-
"""Salesforce/blip2-opt-2.7b.ipynb … modified for target chart size 640-768px"""

!pip install -q transformers torch accelerate pillow openpyxl tqdm

import os, re, json, random
import time
from datetime import datetime
import pandas as pd
import torch
from tqdm import tqdm
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from huggingface_hub import notebook_login
from google.colab import drive


In [2]:
# ---------------- Mount Drive ----------------
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ---------------- Configuration ----------------
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

HF_TOKEN = ""   # your token
MODEL_ID = "Salesforce/blip2-opt-2.7b"
DTYPE = torch.float16

os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
notebook_login()

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Running on {device.upper()}")

✅ Running on CUDA


In [5]:
# ---------------- Paths ----------------
DATA_ROOT = "/content/drive/MyDrive/Project/test_folder"
TF_JSON = os.path.join(DATA_ROOT, "TF_data.json")
MC_JSON = os.path.join(DATA_ROOT, "MC_data.json")

DETAILED_RESULTS_XLSX = os.path.join(DATA_ROOT, "blip2_detailed_results.xlsx")
SUMMARY_RESULTS_XLSX  = os.path.join(DATA_ROOT, "blip2_summary_results.xlsx")

assert os.path.exists(TF_JSON), f"TF JSON not found: {TF_JSON}"
assert os.path.exists(MC_JSON), f"MC JSON not found: {MC_JSON}"

In [6]:
# ---------------- Normalization Helpers ----------------
def normalize_tf(pred):
    p = str(pred).strip().lower()
    if re.fullmatch(r"(true|t)", p): return "True"
    if re.fullmatch(r"(false|f)", p): return "False"
    if "true" in p and "false" not in p: return "True"
    if "false" in p and "true" not in p: return "False"
    return "True" if "yes" in p else ("False" if "no" in p else "False")

def normalize_mc(pred, choices=None):
    if not pred: return ""
    p = str(pred).strip().upper()
    m = re.search(r"\b([A-F])\b", p)
    if m: return m.group(1)
    if choices:
        for idx, c in enumerate(choices):
            if c and c.strip().lower() in p.lower():
                return chr(ord("A") + idx)
    return p[:1] if p else ""

def build_prompt(qtype, question, choices=None):
    if qtype == "TF":
        return (
            "Example 1: Net profit rose in 2020 vs 2019? <answer>False</answer>\n"
            "Example 2: Expenses decreased in 2022 vs 2021? <answer>True</answer>\n"
            "<start_of_image>\n"
            "You are given a chart image and a True/False question.\n"
            "Respond ONLY with 'True' or 'False' inside <answer> tags.\n\n"
            f"Question: {question}\n<answer>"
        )
    elif qtype == "MC":
        example = (
            "Example 1: Which year had highest revenue? Choices: A.2018 B.2019 C.2020 D.2021 <answer>A</answer>\n"
            "Example 2: Which product had lowest cost? Choices: A.P1 B.P2 C.P3 D.P4 <answer>D</answer>\n"
            "Example 3: Which region grew fastest? Choices: A.Asia B.Europe C.US D.Africa <answer>B</answer>\n"
        )
        choices_str = "\n".join(f"{chr(ord('A')+i)}. {c}" for i, c in enumerate(choices or []))
        return (
            example
            + "<start_of_image>\n"
            "You are given a chart image and a multiple‐choice question.\n"
            "Respond ONLY with the letter inside <answer> tags.\n\n"
            f"Question: {question}\nChoices:\n{choices_str}\n<answer>"
        )
    else:
        raise ValueError(f"Unsupported question type: {qtype}")

def extract_final_answer(raw_output: str, choices=None):
    if not isinstance(raw_output, str) or not raw_output.strip():
        return ""
    text = raw_output.strip()
    m_tag = re.findall(r"<\s*answer\s*>\s*([A-Z0-9TrueFalse]+?)\s*<\s*/\s*answer\s*>", text,
                       flags=re.IGNORECASE)
    if m_tag:
        ans = m_tag[-1].strip().capitalize()
        return ans
    if choices:
        for idx, c in enumerate(choices):
            if c and c.strip().lower() in text.lower():
                return chr(ord('A') + idx)
    m_fallback = re.findall(r"\b(True|False|[A-F])\b", text, flags=re.IGNORECASE)
    if m_fallback:
        return m_fallback[-1].strip().capitalize()
    return ""

def load_items(path, qtype, subfolder):
    with open(path, "r", encoding="utf-8") as f:
        items = json.load(f)
    return [
        {
            "type":      qtype,
            "question":  it["question"],
            "answer":    it["answer"],
            "choices":   (it.get("choices") if qtype=="MC" else None),
            "image_path": os.path.join(DATA_ROOT, subfolder, it["image"])
        }
        for it in items
    ]

def load_local_dataset(num_mc=40, num_tf=40):
    mc = load_items(MC_JSON, "MC", "MC_images")
    tf = load_items(TF_JSON, "TF", "TF_images")
    data = mc[:num_mc] + tf[:num_tf]
    random.shuffle(data)
    return data

In [7]:
# ---------------- Model Loading ----------------
print("🔄 Loading BLIP-2 OPT-2.7B …")
processor = Blip2Processor.from_pretrained(MODEL_ID)
model = Blip2ForConditionalGeneration.from_pretrained(
    MODEL_ID,
    torch_dtype=DTYPE,
    device_map="auto"
).eval()
print("✅ Model loaded on:", device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


🔄 Loading BLIP-2 OPT-2.7B …


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

✅ Model loaded on: cuda


In [8]:
# ---------------- Inference with Resize Logic ----------------
@torch.no_grad()
def infer_blip2(image_path, question, qtype, choices=None):
    prompt = build_prompt(qtype, question, choices)
    image = Image.open(image_path).convert("RGB")

    # Print original size
    orig_w, orig_h = image.size
    print(f"🖼️ Original image size: {orig_w}×{orig_h}")

    # Determine target long side
    long_side = max(orig_w, orig_h)
    # Choose target in [640, 768]
    if long_side <= 640:
        target_L = 640
    else:
        target_L = 768

    # Compute scale
    scale = target_L / long_side
    new_w = int(orig_w * scale)
    new_h = int(orig_h * scale)
    # Snap long side to exactly target and maintain ratio
    if orig_w >= orig_h:
        new_w = target_L
        new_h = int(orig_h * scale)
    else:
        new_h = target_L
        new_w = int(orig_w * scale)

    # Optional: round to nearest multiple of 16 for patch grid
    new_w = (new_w // 16) * 16
    new_h = (new_h // 16) * 16

    # Resize image
    image_resized = image.resize((new_w, new_h), Image.LANCZOS)
    print(f"🔧 Resized image size: {new_w}×{new_h}")

    inputs = processor(images=image_resized, text=prompt, return_tensors="pt").to(device, DTYPE)
    out_ids = model.generate(**inputs, max_new_tokens=128)
    answer = processor.decode(out_ids[0], skip_special_tokens=True)
    return answer.strip()


def accuracy(preds, gts):
    return sum(str(p).strip() == str(gt).strip() for p, gt in zip(preds, gts)) / len(preds) if preds else 0

In [9]:
# ---------------- Evaluation ----------------
def evaluate_and_summarize(examples):
    rows, preds, gts, types = [], [], [], []
    print("🚀 Running inference …")
    start_time = time.time()
    for idx, ex in enumerate(tqdm(examples, desc="Evaluating", unit="example")):
        raw = infer_blip2(ex["image_path"], ex["question"], ex["type"], ex.get("choices"))
        if ex["type"] == "MC":
            extracted = extract_final_answer(raw, ex.get("choices"))
        else:
            extracted = extract_final_answer(raw, None)

        print(f"\n📄 Example {idx+1}/{len(examples)}")
        print(f"🖼️ Image: {os.path.basename(ex['image_path'])}")
        print(f"❓ Question: {ex['question']}")
        print(f"RAW OUTPUT: {raw}")
        print(f"EXTRACTED ANSWER: {extracted}\n")

        if ex["type"] == "MC":
            norm = normalize_mc(extracted, ex.get("choices"))
        else:
            norm = normalize_tf(extracted)

        rows.append({
            "image_path": ex["image_path"],
            "type":       ex["type"],
            "question":   ex["question"],
            "ground_truth": ex["answer"],
            "prediction_by_model": norm,
            "raw_output":           (raw[:200] + "..." if len(raw)>200 else raw),
            "raw_output_full":      raw,
            "raw_output_length":    len(raw)
        })
        preds.append(norm)
        gts.append(ex["answer"])
        types.append(ex["type"])

    end_time = time.time()
    total_time = end_time - start_time
    print("✅ Inference completed!")
    print(f"⌛ Total time taken: {total_time:.2f} seconds")

    summary = {
        "model_name":    MODEL_ID,
        "MC_Score":      round(accuracy([p for p,t in zip(preds,types) if t=="MC"],
                                        [g for g,t in zip(gts,types) if t=="MC"]), 4),
        "TF_Score":      round(accuracy([p for p,t in zip(preds,types) if t=="TF"],
                                        [g for g,t in zip(gts,types) if t=="TF"]), 4),
        "Weighted_Avg":  round(accuracy(preds, gts), 4),
        "Total_Questions": len(preds),
        "Total_Time_Sec": total_time,
        "Timestamp":      datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    print(f"📊 MC Accuracy: {summary['MC_Score']}")
    print(f"📊 TF Accuracy: {summary['TF_Score']}")
    print(f"📊 Weighted Avg: {summary['Weighted_Avg']}")
    return pd.DataFrame(rows), summary


In [10]:
# ---------------- Main Execution ----------------
if __name__ == "__main__":
    examples = load_local_dataset(num_mc=40, num_tf=40)
    print(f"Total examples: {len(examples)}")
    df_details, summary = evaluate_and_summarize(examples)

    timestamp     = datetime.now().strftime("%Y%m%d_%H%M%S")
    detailed_path = DETAILED_RESULTS_XLSX.replace(".xlsx", f"_{timestamp}.xlsx")
    df_details.to_excel(detailed_path, index=False)
    print("✅ Saved detailed results:", detailed_path)

    df_sum = pd.DataFrame([summary])
    if os.path.exists(SUMMARY_RESULTS_XLSX):
        old   = pd.read_excel(SUMMARY_RESULTS_XLSX)
        df_sum = pd.concat([old, df_sum], ignore_index=True)
    df_sum.to_excel(SUMMARY_RESULTS_XLSX, index=False)
    print("✅ Saved summary results:", SUMMARY_RESULTS_XLSX)

Total examples: 80
🚀 Running inference …


Evaluating:   0%|          | 0/80 [00:00<?, ?example/s]

🖼️ Original image size: 1652×889
🔧 Resized image size: 768×400


Evaluating:   1%|▏         | 1/80 [00:02<03:03,  2.32s/example]


📄 Example 1/80
🖼️ Image: 1245561843_12_crop_0.jpg
❓ Question: The occupancy rate for Assisted Living decreased from 1Q12 to 3Q14.
RAW OUTPUT: Example 1: Net profit rose in 2020 vs 2019? <answer>False</answer>
Example 2: Expenses decreased in 2022 vs 2021? <answer>True</answer>
<start_of_image>
You are given a chart image and a True/False question.
Respond ONLY with 'True' or 'False' inside <answer> tags.

Question: The occupancy rate for Assisted Living decreased from 1Q12 to 3Q14.
<answer>False</answer>
EXTRACTED ANSWER: False



...

Evaluating:  99%|█████████▉| 79/80 [00:34<00:00,  5.00example/s]


📄 Example 78/80
🖼️ Image: 1246868763_15_crop_0.jpg
❓ Question: Which brand experienced the largest increase in units from 2009 to Q3 2014 TTM?
RAW OUTPUT: Example 1: Which year had highest revenue? Choices: A.2018 B.2019 C.2020 D.2021 <answer>A</answer>
Example 2: Which product had lowest cost? Choices: A.P1 B.P2 C.P3 D.P4 <answer>D</answer>
Example 3: Which region grew fastest? Choices: A.Asia B.Europe C.US D.Africa <answer>B</answer>
<start_of_image>
You are given a chart image and a multiple‐choice question.
Respond ONLY with the letter inside <answer> tags.

Question: Which brand experienced the largest increase in units from 2009 to Q3 2014 TTM?
Choices:
A. A
B. B
C. C
D. D
<answer>C</answer>
EXTRACTED ANSWER: C

🖼️ Original image size: 891×627
🔧 Resized image size: 768×528

📄 Example 79/80
🖼️ Image: 1245561305_57_crop_0.jpg
❓ Question: Which method had the largest difference between the percentage of respondents who considered it but did not use it and those who considered and f

Evaluating: 100%|██████████| 80/80 [00:34<00:00,  2.30example/s]

🖼️ Original image size: 1872×802
🔧 Resized image size: 768×320

📄 Example 80/80
🖼️ Image: 1245866113_33_crop_0.jpg
❓ Question: Which product achieved the highest total unadjusted product revenue by the fifth quarter after launch?
RAW OUTPUT: Example 1: Which year had highest revenue? Choices: A.2018 B.2019 C.2020 D.2021 <answer>A</answer>
Example 2: Which product had lowest cost? Choices: A.P1 B.P2 C.P3 D.P4 <answer>D</answer>
Example 3: Which region grew fastest? Choices: A.Asia B.Europe C.US D.Africa <answer>B</answer>
<start_of_image>
You are given a chart image and a multiple‐choice question.
Respond ONLY with the letter inside <answer> tags.

Question: Which product achieved the highest total unadjusted product revenue by the fifth quarter after launch?
Choices:
A. A
B. B
C. C
D. D
<answer>C</answer>
EXTRACTED ANSWER: C

✅ Inference completed!
⌛ Total time taken: 34.82 seconds
📊 MC Accuracy: 0.225
📊 TF Accuracy: 0.525
📊 Weighted Avg: 0.375


✅ Saved detailed results: /content/drive/MyDrive/Project/test_folder/blip2_detailed_results_20251110_132656.xlsx
✅ Saved summary results: /content/drive/MyDrive/Project/test_folder/blip2_summary_results.xlsx


### Summary of 5 Sample Examples (including evaluation)

Example 1
- Image: 1245561305_57_crop_0.jpg
- Original image size: 891 × 627 pixels
- Resized image size: 768 × 528 pixels
- About 2.05 seconds per example


Example 2
- Image: same file 1245561305_57_crop_0.jpg
- Original size: 891 × 627
- Resized size: 768 × 528
- ~1.2 seconds each

Example 3
- Image: same file 1245561305_57_crop_0.jpg
- Original size: 891 × 627
- Resized size: 768 × 528
- 1.24 seconds per example.

Example 4
- Image: 1245561305_57_crop_0.jpg
- Original size: 882 × 997
- Resized size: 672 × 768 (because the long side was 997 so the target was 768)
- ~1.07 seconds per example

Example 5
- Image: 1243210261_13_crop_0.jpg
- Original size: 882 × 997
- Resized size: 672 × 768
- ~1.46 seconds per example


